# Solutions for Lab Sheet 6 (COM3502-4502-6502 Speech Processing)

This lab sheet (solution) is part of the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield").

It is probably easiest to open this Jupyter Notebook with [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true "Open in Google Colab") since GitHub's Viewer does not always show all details correctly. <a href="https://colab.research.google.com/github/sap-shef/SpeechProcesssingLab/blob/main/Lab-Sheet-Solutions/Lab-Sheet-6-Solution.ipynb"><img align="right" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open Notebook in Google Colab" title="Open and Execute the Notebook directly in Google Colaboratory"></a>

Please put questions, comments and correction suggestions in the [Blackboard](https://vle.shef.ac.uk) discussion board or send an email to [s.goetze@sheffield.ac.uk](mailto:s.goetze@sheffield.ac.uk).

In [ ]:
# Let's do the ususal necessary and nice-to-have imports
%matplotlib inline
import matplotlib.pyplot as plt  # plotting
import seaborn as sns; sns.set() # styling ((un-)comment if you want)
import numpy as np               # math

## The Chirp Signal, a.k.a. Sweep Signal

The function 
\begin{equation}
x_{\mathrm{chirp}}(t)=\sin\left(\pi t^2\right) \tag{1}
\end{equation}
defines the so-called **linear-frequency [chirp](https://en.wikipedia.org/wiki/Chirp "Click here for additional information on the Chirp signal on Wikipedia")** or simply linear chirp. It is a signal that sweeps through diffrent frequencies over time.

The instantaneous frequency $f(t)$ of this chirp signal increases exactly linearly with time, i.e. $f(t)=f_0+ a t$ with a constant $a=\frac{f_1-f_0}{t_1-t_0}$. More precisely, the instantaneous angular frequency $\omega(t)$ of the chirp signal at time $t$ is the derivate of the sinusoid's argument divided by $2\pi$, thus $\omega(t) = t$. 

In [ ]:
fs=8000      # sampling frequency
t1=0         # time 1 in seconds
t2=12        # time 2 in seconds
length=t2-t1 # signlal length in seconds

# create time vector from t1 to t2
t = np.linspace(t1, t2, length*fs) # time vector from t1 to t2
# create chirp signal
x = np.sin(np.pi*t**2)

# plot chirp signal
plt.figure()
plt.subplot(2,1,1)
plt.plot(t,x)
plt.ylabel('$x_{\mathrm{chirp}}(t)$')
plt.xlabel('$t$ in sec')
plt.title('Linear Chirp $x_{\mathrm{chirp}}(t)=\mathrm{sin}(\pi t^2)$ between t='+str(t1)+' sec and t='+str(t2)+' sec')

#spectrum here (for different times)

plt.tight_layout()

### Spectrogram vs. Spectrum of Chirp Signal

In [ ]:
plt.subplot(2,1,2)
L = 9192        # DFT length (we need a relatively large number here for high frequency resolution)
overlap = 4096  # also large overlap to get some time resolution between segments
plt.specgram(x,NFFT=L, Fs=fs, noverlap=overlap)
plt.grid(False)
plt.ylim(0,50)
plt.colorbar()  # add a colorbar to the spectrogram
plt.clim(-100,0)
plt.title('Spectrogram of Linear Chirp')
plt.ylabel('$f$ in Hz')
plt.xlabel('$t$ in sec')
None

### Colorbars for Spectrograms and Controlling the Colorbar
Using the module `gridspec` allows to align plots that have elements with and without a colorbar. This is demonstrated by the following example, which uses the gridspec-constructor with keywords specified by the dictionary gridspec_kw. This dictionary, in turn, is one of the parameters of plt.subplots, which creates a figure and a set of subplots.

In [ ]:
fig, ax = plt.subplots(2, 2, gridspec_kw={'width_ratios':  [1, 0.05], 
                                          'height_ratios': [1, 2]}, figsize=(8, 4))
im = ax[0, 0].plot(t, x, color='gray')
ax[0, 0].set_xlabel('Time (seconds)')
ax[0, 0].set_ylabel('Amplitude')

ax[0, 0].set_xlim([t1, t2])
#ax[0, 0].set_ylim([-0.40, 0.40])
ax[0, 1].set_axis_off() # do not display the axis in right upper corner


spec,_,_,im = ax[1, 0].specgram(x,NFFT=L, Fs=fs, noverlap=overlap)
#im = ax[1, 0].imshow(10*np.log10(spec), aspect='auto', origin='lower', extent=[t1, t2, 0, 50])
plt.colorbar(mappable=im, cax=ax[1, 1])
ax[1, 0].set_xlabel('Time (seconds)')
ax[1, 0].set_ylabel('$f$ in Hz')
ax[1, 0].grid(False)
ax[1, 1].set_ylabel('Magnitude')

plt.tight_layout()

## Overlap-Add

In [ ]:
def nextPowerOf2(n):
    '''
    Calculates the smallest power of 2 which is bigger than input variable n
    
    Example:
        for n in range(20):
            print('nextPowerOf2(n) for n='+str(n)+' is '+str(nextPowerOf2(n)))
    '''
    if (n<2):
        return 2
    # If n is a power of 2 then return n 
    if (n and not(n & (n - 1))):
        return n
    # If n is NOT a power of 2 
    p = 1
    while (p < n) :
        p <<= 1 
    return p
 


L_x = 32
L_h = 4
x=np.ones(L_x)
h=np.ones(L_h)

L_Bl  = 8   # block length (see Fig 2.25 in https://staffwww.dcs.shef.ac.uk/people/S.Goetze/book/Ch2.S3.html)
L_hop = 8
L_DFT = nextPowerOf2(L_Bl+L_h-1)
# L_DFT = 16

noOfFrames = (L_x // L_hop) # number of blocks

# outer loop over mic channels
# h = np.zeros(L_DFT)
# h[:L_h] = h
H = np.fft.rfft(h, n=L_DFT)
oldBlock = []
for iFrame in range(noOfFrames):
  #x_block = np.zeros(L_DFT) # zero padding # not needed
  k = np.arange(iFrame*L_hop, iFrame*L_hop+L_Bl) # time vector
  x_block = x[k]
  # k = np.hstack((k,np.zeros(L_DFT-len(k))))  # zero padding
  X_block = np.fft.rfft(x_block, n=L_DFT)

  Y = X_block*H # frequency domain multiplication
  y = np.fft.irfft(Y, n=L_DFT)

  k2 = np.arange(iFrame*L_hop, iFrame*L_hop+L_DFT) # time vector
  plt.figure()
  plt.stem(k2,y)